In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os 
import matplotlib.pyplot as plt
import time

In [2]:
df = pd.read_csv(r'C:\Users\SONY\Desktop\Machine Learning\Tensorflow\datasets\train.csv')

In [3]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
df.shape

(7613, 5)

In [13]:
import re
import string 
def remove_Url(text):
    url =  re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)
def remove_punc(text):
    translator = str.maketrans("" , "",string.punctuation)
    return text.translate(translator)
string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
df['text'] = df.text.map(remove_Url)


In [15]:
df['text'] = df.text.map(remove_punc)

In [27]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop  =  set(stopwords.words("english"))
def remo_w(text):
    filters  = (word.lower() for word in text.split() if word.lower() not in stop)
    return " ".join(filters)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SONY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
stop

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [29]:
df['text'] =df.text.map(remo_w)

In [40]:
from collections import Counter
def counter_s(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count

counter =  counter_s(df.text)

In [41]:
len(counter)

17971

In [42]:
counter.most_common(5)

[('like', 345), ('im', 299), ('amp', 298), ('fire', 250), ('get', 229)]

In [43]:
num_unique_word = len(counter)

In [44]:
num_unique_word

17971

In [45]:
train_size = int(df.shape[0] * 0.8)

train_df = df[:train_size]
val_df = df[train_size:]

# split text and labels
train_sentences = train_df.text.to_numpy()
train_labels = train_df.target.to_numpy()
val_sentences = val_df.text.to_numpy()
val_labels = val_df.target.to_numpy()

In [47]:
from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence of integers
tokenizer = Tokenizer(num_words=num_unique_word)
tokenizer.fit_on_texts(train_sentences) # fit only to training

In [49]:
word_index = tokenizer.word_index

In [50]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [51]:
# Pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Max number of words in a sequence
max_length = 20

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, val_padded.shape

((6090, 20), (1523, 20))

In [52]:
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [54]:
#  Create LSTM model
from tensorflow.keras import layers

# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

# Word embeddings give us a way to use an efficient, dense representation in which similar words have 
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a 
# dense vector of floating point values (the length of the vector is a parameter you specify).

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_word, 32, input_length=max_length))

# The layer will take as input an integer matrix of size (batch, input_length),
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.


model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 32)            575072    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 599,969
Trainable params: 599,969
Non-trainable params: 0
_________________________________________________________________


In [55]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

C:\Users\SONY\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [57]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [58]:
model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=2)

Epoch 1/20
191/191 - 16s - loss: 0.5489 - accuracy: 0.7112 - val_loss: 0.4701 - val_accuracy: 0.7774
Epoch 2/20
191/191 - 6s - loss: 0.2920 - accuracy: 0.8837 - val_loss: 0.4930 - val_accuracy: 0.7794
Epoch 3/20
191/191 - 6s - loss: 0.1591 - accuracy: 0.9488 - val_loss: 0.5507 - val_accuracy: 0.7761
Epoch 4/20
191/191 - 6s - loss: 0.1100 - accuracy: 0.9665 - val_loss: 0.8361 - val_accuracy: 0.7544
Epoch 5/20
191/191 - 6s - loss: 0.0873 - accuracy: 0.9744 - val_loss: 0.9581 - val_accuracy: 0.7446
Epoch 6/20
191/191 - 6s - loss: 0.0742 - accuracy: 0.9749 - val_loss: 0.6686 - val_accuracy: 0.7433
Epoch 7/20
191/191 - 6s - loss: 0.0589 - accuracy: 0.9777 - val_loss: 0.9236 - val_accuracy: 0.7354
Epoch 8/20
191/191 - 6s - loss: 0.0489 - accuracy: 0.9808 - val_loss: 1.0303 - val_accuracy: 0.7439
Epoch 9/20
191/191 - 6s - loss: 0.0442 - accuracy: 0.9803 - val_loss: 0.9538 - val_accuracy: 0.7426
Epoch 10/20
191/191 - 6s - loss: 0.0386 - accuracy: 0.9834 - val_loss: 1.3206 - val_accuracy: 0.736

In [59]:

predictions = model.predict(train_padded)
predictions = [1 if p > 0.5 else 0 for p in predictions]

In [60]:
print(train_sentences[10:20])

print(train_labels[10:20])
print(predictions[10:20])

['three people died heat wave far'
 'haha south tampa getting flooded hah wait second live south tampa gonna gonna fvck flooding'
 'raining flooding florida tampabay tampa 18 19 days ive lost count'
 'flood bago myanmar arrived bago'
 'damage school bus 80 multi car crash breaking' 'whats man' 'love fruits'
 'summer lovely' 'car fast' 'goooooooaaaaaal']
[1 1 1 1 1 0 0 0 0 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
